# 🔍 DoH Watcher - Standard DNSとの比較対応版
DNS over HTTPS (DoH) の効果を可視化・比較する教育向けツール。

このノートブックでは、Google DoH、Cloudflare DoH、そして通常のDNSを使ってDNSクエリを実行し、応答時間を比較します。


In [ ]:
# 検証対象ドメインとクエリ回数
domains = ['example.com', 'google.com', 'cloudflare.com']
query_count = 5


In [ ]:
import requests
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import socket


In [ ]:
DOH_GOOGLE = "https://dns.google/resolve"
DOH_CLOUDFLARE = "https://cloudflare-dns.com/dns-query"

def resolve_doh(domain, provider='google'):
    try:
        if provider == 'google':
            start = time.time()
            r = requests.get(DOH_GOOGLE, params={'name': domain, 'type': 'A'})
            elapsed = (time.time() - start) * 1000
        elif provider == 'cloudflare':
            headers = {'accept': 'application/dns-json'}
            start = time.time()
            r = requests.get(DOH_CLOUDFLARE, params={'name': domain, 'type': 'A'}, headers=headers)
            elapsed = (time.time() - start) * 1000
        if r.status_code == 200:
            return elapsed
        else:
            return None
    except Exception:
        return None

def resolve_standard_dns(domain):
    try:
        start = time.time()
        socket.gethostbyname(domain)
        elapsed = (time.time() - start) * 1000
        return elapsed
    except Exception:
        return None


In [ ]:
results = []

for domain in domains:
    for i in range(query_count):
        elapsed_std = resolve_standard_dns(domain)
        elapsed_g = resolve_doh(domain, provider='google')
        elapsed_c = resolve_doh(domain, provider='cloudflare')
        results.append({'domain': domain, 'provider': 'Standard DNS', 'elapsed_ms': elapsed_std})
        results.append({'domain': domain, 'provider': 'Google DoH', 'elapsed_ms': elapsed_g})
        results.append({'domain': domain, 'provider': 'Cloudflare DoH', 'elapsed_ms': elapsed_c})

df = pd.DataFrame(results)
df.head()


In [ ]:
avg_df = df.groupby(['domain', 'provider'])['elapsed_ms'].mean().reset_index()

sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.barplot(data=avg_df, x='domain', y='elapsed_ms', hue='provider')
plt.title("DNS Query Response Time Comparison (Standard DNS vs DoH)")
plt.ylabel("Average Response Time (ms)")
plt.xlabel("Domain")
plt.tight_layout()
plt.show()


## 🧠 学びのポイント

- 通常のDNS（UDP 53番）は速いが、通信内容が暗号化されていないため第三者に見える。
- DoHはHTTPS通信として送るため、内容が秘匿されプライバシーが守られる。
- DoH導入によりやや遅延があるが、セキュリティとトレードオフの関係にある。
- 通信の種類や目的、環境に応じた選択が重要。
